In [ ]:
#!pip install pillow

In [ ]:
import numpy as np
import pandas as pd

filename = f'../npz/BTCUSD_1T.npz'
data = np.load(filename, allow_pickle=True)['data']
# преобразование в формат pandas
loaded_df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

In [ ]:
# Преобразование 'timestamp' в datetime и установка его как индекс DataFrame
loaded_df['timestamp'] = pd.to_datetime(loaded_df['timestamp'])
loaded_df.set_index('timestamp', inplace=True)

# Преобразование всех столбцов с ценами и объемом в числовой формат
for column in ['open', 'high', 'low', 'close', 'volume']:
    loaded_df[column] = pd.to_numeric(loaded_df[column], errors='coerce')

In [ ]:
from PIL import Image

def generate_image_from_matrix(matrix):
    """
    Функция для создания и отображения изображения из двумерной матрицы целых чисел.
    Значение элемента матрицы определяет яркость пикселя.
    
    :param matrix: двумерный массив numpy с целыми числами от 0 до 255
    """
    # Проверим, что матрица содержит только значения от 0 до 255
    if not (np.issubdtype(matrix.dtype, np.integer) and matrix.min() >= 0 and matrix.max() <= 255):
        raise ValueError("Матрица должна содержать целые числа в диапазоне от 0 до 255.")
    
    # Создаем изображение
    image = Image.fromarray(matrix.astype('uint8'), mode='L')
    
    # Отображаем изображение в результатах Jupyter Notebook
    display(image)

In [ ]:
# select last 10000 rows
df = loaded_df.tail(50000)
# df = loaded_df
df.shape

In [ ]:
# calculate the moving average for close prices
ma1 = df['close'].rolling(window=30).mean()
ma2 = df['close'].rolling(window=60).mean()
ma3 = df['close'].rolling(window=120).mean()
ma4 = df['close'].rolling(window=240).mean()

In [ ]:
ma2_shifted = ma2.shift(-15)
ma3_shifted = ma3.shift(-30)
ma4_shifted = ma4.shift(-60)

In [ ]:
import matplotlib.pyplot as plt
n = -3000

plt.figure(figsize=(15, 7))

#plt.plot(ma1[n:], color='red')
#plt.plot(ma2.shift(-15)[n:], color='blue')
#plt.plot(ma2[n:], color='red')
#plt.plot(ma3.shift(-30)[n:], color='blue')
plt.plot(ma3[n:], color='red')
plt.plot(ma4.shift(-60)[n:], color='blue')
plt.show()

In [ ]:
# calculate difference between 10MA and 20MA
x = (ma1 - ma2_shifted).fillna(0)
y = (ma2 - ma3_shifted).fillna(0)
z = (ma3_shifted - ma4_shifted).fillna(0)
plt.plot(x[n:], y[n:])

In [ ]:
for i in range(0, 20):
    w=i*1
    print(w)
    plt.figure(figsize=(12, 6))
    plt.plot((df['close'].rolling(window=30).mean().shift(-15) - df['close'].rolling(window=30+2*w).mean().shift(-15-w))[-3000:])
    plt.show()

In [ ]:
import sys
import os

# Get the current working directory
current_dir = os.getcwd()

# Get the parent directory
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))

# Add the parent directory to sys.path
sys.path.append(parent_dir)

# # Now you can import the module
# from parent_directory import module_to_import

# # Use the imported module
# module_to_import.some_function()
from tsl import get_trailing_stop_loss_long_profits, get_trailing_stop_loss_short_profits

(long_profits, long_stop_loss_prices, maximums) = get_trailing_stop_loss_long_profits(df, 0.9 * 0.01)
(short_profits, short_stop_loss_prices, minimums) = get_trailing_stop_loss_short_profits(df, 0.5 * 0.01)

In [ ]:
long_profits_df = pd.DataFrame(long_profits)
long_profits_df.ffill(inplace=True)
# long_profits_df
short_profits_df = pd.DataFrame(short_profits)
short_profits_df.ffill(inplace=True)

In [ ]:
import plotly.graph_objs as go

# Create a 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='lines',
    line=dict(
        #color='blue',
        color=(short_profits_df+long_profits_df).to_numpy().flatten(),
        colorscale='Viridis',  # Choose a colorscale
        width=2
    ),
    marker=dict(
        size=2,
        color=(short_profits_df+long_profits_df).to_numpy().flatten(),
        colorscale='Viridis',  # Choose a colorscale
        opacity=0.2
    )
)])

# Update layout for better visualization
fig.update_layout(
    title='3D Scatter Plot',
    scene=dict(
        xaxis_title='X Axis',
        yaxis_title='Y Axis',
        zaxis_title='Z Axis'
    ),
    width=800,
    height=600
)

# Show the plot
fig.show()

In [ ]:
def get_density(matrix_size = 300):
    matrix = np.zeros((matrix_size, matrix_size), dtype=np.float32)
    # loop over x and increment the value of the matrix
    min_x = x.min()
    max_x = x.max()
    min_y = y.min()
    max_y = y.max()
    width = max_x - min_x
    height = max_y - min_y
    last_i, last_j = -1, -1
    for k in range(len(x)):
        i = int((x.iloc[k] - min_x) / width * (matrix_size - 1))
        j = int((y.iloc[k] - min_y) / height * (matrix_size - 1))
        if (i != last_i) or (j != last_j):
            matrix[j, i] += 1
            last_i, last_j = i, j
    return matrix

In [ ]:
matrix = get_density()

In [ ]:
matrix_log = np.log(matrix + 1)

In [ ]:
def rescale_matrix(matrix, new_min=0, new_max=255):
    # Find the minimum and maximum values in the matrix
    old_min = np.min(matrix)
    old_max = np.max(matrix)
    
    # Rescale the matrix to the new range [new_min, new_max]
    rescaled_matrix = (matrix - old_min) / (old_max - old_min) * (new_max - new_min) + new_min
    
    # Convert to the desired dtype (e.g., uint8)
    rescaled_matrix = np.clip(rescaled_matrix, new_min, new_max).astype(np.uint8)
    
    return rescaled_matrix

In [ ]:
resc = rescale_matrix(matrix_log)

In [ ]:
generate_image_from_matrix(resc)

In [ ]:
#!pip install seaborn

In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 10))
# sns.heatmap(matrix_log[130:260,130:260], cmap='viridis')
sns.heatmap(matrix_log, cmap='viridis')
plt.show()

In [ ]:
row = matrix[198]
row[180:220]

In [ ]:
# plot row
plt.plot(row)